<a href="https://colab.research.google.com/github/PyBeginner1/Gym-Crowd-Size-Prediction/blob/main/Gym_Crowd_Size_Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [113]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

from sklearn.linear_model import Ridge
from sklearn.neural_network import MLPRegressor
from sklearn.ensemble import RandomForestRegressor

from sklearn import metrics

import warnings
warnings.filterwarnings(action='ignore')

In [114]:
data = pd.read_csv('/content/data[2].csv')

In [115]:
data

,number_people,date,timestamp,day_of_week,is_weekend,is_holiday,temperature,is_start_of_semester,is_during_semester,month,hour
0,37,2015-08-14 17:00:11-07:00,61211,4,0,0,71.76,0,0,8,17
1,45,2015-08-14 17:20:14-07:00,62414,4,0,0,71.76,0,0,8,17
2,40,2015-08-14 17:30:15-07:00,63015,4,0,0,71.76,0,0,8,17
3,44,2015-08-14 17:40:16-07:00,63616,4,0,0,71.76,0,0,8,17
4,45,2015-08-14 17:50:17-07:00,64217,4,0,0,71.76,0,0,8,17
...,...,...,...,...,...,...,...,...,...,...,...
62179,23,2017-03-18 18:42:28-07:00,67348,5,1,0,61.07,0,1,3,18
62180,21,2017-03-18 18:52:35-07:00,67955,5,1,0,61.07,0,1,3,18
62181,25,2017-03-18 19:02:40-07:00,68560,5,1,0,56.71,0,1,3,19
62182,18,2017-03-18 19:12:47-07:00,69167,5,1,0,56.71,0,1,3,19


In [116]:
data.isna().sum()

number_people           0
date                    0
timestamp               0
day_of_week             0
is_weekend              0
is_holiday              0
temperature             0
is_start_of_semester    0
is_during_semester      0
month                   0
hour                    0
dtype: int64

In [117]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 62184 entries, 0 to 62183
Data columns (total 11 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   number_people         62184 non-null  int64  
 1   date                  62184 non-null  object 
 2   timestamp             62184 non-null  int64  
 3   day_of_week           62184 non-null  int64  
 4   is_weekend            62184 non-null  int64  
 5   is_holiday            62184 non-null  int64  
 6   temperature           62184 non-null  float64
 7   is_start_of_semester  62184 non-null  int64  
 8   is_during_semester    62184 non-null  int64  
 9   month                 62184 non-null  int64  
 10  hour                  62184 non-null  int64  
dtypes: float64(1), int64(9), object(1)
memory usage: 5.2+ MB


In [118]:
pd.to_datetime(data['date'])

0        2015-08-14 17:00:11-07:00
1        2015-08-14 17:20:14-07:00
2        2015-08-14 17:30:15-07:00
3        2015-08-14 17:40:16-07:00
4        2015-08-14 17:50:17-07:00
                   ...            
62179    2017-03-18 18:42:28-07:00
62180    2017-03-18 18:52:35-07:00
62181    2017-03-18 19:02:40-07:00
62182    2017-03-18 19:12:47-07:00
62183    2017-03-18 19:22:51-07:00
Name: date, Length: 62184, dtype: object

In [119]:
pd.to_datetime('2015-08-14 17:00:11-07:00').hour

17

In [120]:
pd.to_datetime('2015-08-14 17:00:11-07:00').minute

0

In [121]:
pd.to_datetime('2015-08-14 17:00:11-07:00').second

11

#Preprocessing

In [122]:
def preprocess_inputs(df):
  df = df.copy()

  #extract date
  df['date'] = pd.to_datetime(df['date'])
  df['month'] = df['date'].apply(lambda x: x.month)
  df['day'] = df['date'].apply(lambda x: x.day)
  df['hour'] = df['date'].apply(lambda x: x.hour)
  df['minute'] = df['date'].apply(lambda x: x.minute)
  df = df.drop('date', axis = 1 )

  #split X & Y
  X = df.drop('number_people', axis = 1)
  Y = df['number_people']

  #scaling
  scaler = StandardScaler()
  X = pd.DataFrame(scaler.fit_transform(X), columns = X.columns)

  #train-test split
  X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.3, shuffle = True, random_state = 1)

  return X_train, X_test, Y_train, Y_test

In [123]:
X_train, X_test, Y_train, Y_test = preprocess_inputs(data)

In [124]:
X

,timestamp,day_of_week,is_weekend,is_holiday,temperature,is_start_of_semester,is_during_semester,month,hour,day,minute
0,0.636550,0.509561,-0.628051,-0.05079,2.090274,-0.292535,-1.393938,0.162604,0.709116,-0.214711,-1.659141
1,0.686238,0.509561,-0.628051,-0.05079,2.090274,-0.292535,-1.393938,0.162604,0.709116,-0.214711,-0.504656
2,0.711061,0.509561,-0.628051,-0.05079,2.090274,-0.292535,-1.393938,0.162604,0.709116,-0.214711,0.072586
3,0.735885,0.509561,-0.628051,-0.05079,2.090274,-0.292535,-1.393938,0.162604,0.709116,-0.214711,0.649828
4,0.760708,0.509561,-0.628051,-0.05079,2.090274,-0.292535,-1.393938,0.162604,0.709116,-0.214711,1.227071
...,...,...,...,...,...,...,...,...,...,...,...
62179,0.890029,1.010360,1.592228,-0.05079,0.397840,-0.292535,0.717392,-1.288758,0.857979,0.239251,0.765277
62180,0.915100,1.010360,1.592228,-0.05079,0.397840,-0.292535,0.717392,-1.288758,0.857979,0.239251,1.342519
62181,0.940089,1.010360,1.592228,-0.05079,-0.292433,-0.292535,0.717392,-1.288758,1.006842,0.239251,-1.543692
62182,0.965160,1.010360,1.592228,-0.05079,-0.292433,-0.292535,0.717392,-1.288758,1.006842,0.239251,-0.966450


In [125]:
Y

0        37
1        45
2        40
3        44
4        45
         ..
62179    23
62180    21
62181    25
62182    18
62183    23
Name: number_people, Length: 62184, dtype: int64

In [126]:
print(X.shape, X_train.shape, X_test.shape)

(62184, 11) (43528, 11) (18656, 11)


#Model

In [127]:
models = {
    'Linear Regression Ridge': Ridge(),
    'Neural Network' : MLPRegressor(),
    'Random Forest' : RandomForestRegressor()
}

for name, model in models.items():
  model.fit(X_train, Y_train)
  print(name + ' trained!')

Linear Regression Ridge trained!
Neural Network trained!
Random Forest trained!


In [128]:
 from sklearn.metrics import r2_score

In [129]:
for name, model in models.items():
  y_pred = model.predict(X_test)
  score = r2_score(Y_test, y_pred)
  print(name + ' R2 score is:{:.2f}'.format(score))

Linear Regression Ridge R2 score is:0.51
Neural Network R2 score is:0.73
Random Forest R2 score is:0.91


In [130]:
def get_rmse(y_test, y_pred):
    rmse = np.sqrt(np.mean((y_test - y_pred)**2))
    return rmse

for name, model in models.items():
    y_pred = model.predict(X_test)
    rmse = get_rmse(Y_test, y_pred)
    print(name + " RMSE: {:.2f}".format(rmse))

Linear Regression Ridge RMSE: 16.04
Neural Network RMSE: 11.95
Random Forest RMSE: 6.65


In [131]:
#taking data with val result 37
input_data = (0.636550,	0.509561,	-0.628051	,-0.05079	,2.090274,	-0.292535,	-1.393938,	0.162604,	0.709116,	-0.214711,	-1.659141)

#numpy
input_data_array1 = np.asarray(input_data)

#reshape
input_data_reshape1 = input_data_array1.reshape(1, -1)

prediction1 = model.predict(input_data_reshape1)
prediction1

array([40.01])

In [134]:
#for 45
input_data1 = (0.686238,	0.509561,	-0.628051,	-0.05079,	2.090274,	-0.292535,	-1.393938	,0.162604,	0.709116,	-0.214711,	-0.504656)

#numpy
input_data_array2 = np.asarray(input_data1)

#reshape
input_data_reshape2 = input_data_array2.reshape(1, -1)

prediction2 = model.predict(input_data_reshape2)
prediction2

array([43.54])